In [1]:
%pip install -q transformers datasets evaluate torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [38]:
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import tqdm
import evaluate
from torchinfo import summary
from datasets import Dataset, Features, Value, ClassLabel
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
tr = Dataset.from_csv(
    path_or_paths="/content/train.csv",
    features=Features({"text": Value("string"), "rate": ClassLabel(num_classes=5, names=["1", "2", "3", "4", "5"])})).rename_column("rate", "labels")
te = Dataset.from_csv(
    path_or_paths="/content/test.csv",
    features=Features({"text": Value("string"), "index": Value("int64")}))
tr, te

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

(Dataset({
     features: ['text', 'labels'],
     num_rows: 48665
 }),
 Dataset({
     features: ['text', 'index'],
     num_rows: 12167
 }))

In [4]:
spl = tr.train_test_split(test_size=0.15, shuffle=True, stratify_by_column="labels")
spl

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 41365
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 7300
    })
})

In [5]:
model_name = "Tochka-AI/ruRoPEBert-e5-base-2k"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
tok_spl = spl.map(preprocess_function, batched=True)
tok_te = te.map(preprocess_function, batched=True)

Map:   0%|          | 0/41365 [00:00<?, ? examples/s]

Map:   0%|          | 0/7300 [00:00<?, ? examples/s]

Map:   0%|          | 0/12167 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average="micro")

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           trust_remote_code=True,
                                                           attn_implementation='sdpa',
                                                           num_labels=5)

config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

modeling_rope_bert.py:   0%|          | 0.00/46.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Tochka-AI/ruRoPEBert-e5-base-2k:
- modeling_rope_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/556M [00:00<?, ?B/s]

Some weights of RoPEBertForSequenceClassification were not initialized from the model checkpoint at Tochka-AI/ruRoPEBert-e5-base-2k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
for param in model.bert.parameters():
    param.requires_grad = False

In [13]:
model.classifier = nn.Sequential(
    nn.Linear(768, 128, bias=False),
    nn.BatchNorm1d(128),
    nn.Dropout(0.2),
    nn.Linear(128, 5)
)

In [14]:
model

RoPEBertForSequenceClassification(
  (bert): RoPEBertModel(
    (embeddings): RoPEBertEmbeddings(
      (word_embeddings): Embedding(69382, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RoPEBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x RoPEBertLayer(
          (attention): RoPEBertAttention(
            (self): RoPEBertSdpaAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (rotary_emb): BertRotaryEmbedding()
            )
            (output): RoPEBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNo

In [15]:
# Calculate model parameters memory
param_memory = sum(p.numel() for p in model.parameters()) * 4  # in bytes for float32
param_memory_MB = param_memory / (1024 ** 2)  # in MB
print(f"Model Parameters Memory: {param_memory_MB} MB")

Model Parameters Memory: 528.1118354797363 MB


In [16]:
# Gradients memory is the same as parameters
gradients_memory_MB = sum(p.numel() for p in model.parameters() if p.requires_grad) * 4 / (1024 ** 2)
print(f"Gradients Memory: {gradients_memory_MB} MB")

Gradients Memory: 0.3784370422363281 MB


In [17]:
# Adjust based on optimizer type (assuming Adam optimizer here)
optimizer_factor = 2  # 1 for SGD w/ momentum, 2 for Adam
optimizer_memory_MB = gradients_memory_MB * optimizer_factor
print(f"Optimizer Memory: {optimizer_memory_MB} MB")

Optimizer Memory: 0.7568740844726562 MB


In [20]:
from torchinfo import summary

summary(model, input_size=(32, 500), dtypes=[torch.int64])

Layer (type:depth-idx)                                            Output Shape              Param #
RoPEBertForSequenceClassification                                 [32, 5]                   --
├─RoPEBertModel: 1-1                                              [32, 768]                 --
│    └─RoPEBertEmbeddings: 2-1                                    [32, 500, 768]            --
│    │    └─Embedding: 3-1                                        [32, 500, 768]            (53,285,376)
│    │    └─Embedding: 3-2                                        [32, 500, 768]            (768)
│    │    └─LayerNorm: 3-3                                        [32, 500, 768]            (1,536)
│    │    └─Dropout: 3-4                                          [32, 500, 768]            --
│    └─RoPEBertEncoder: 2-2                                       [32, 500, 768]            --
│    │    └─ModuleList: 3-5                                       --                        (85,054,464)
│    └─RoPEBertMe

In [21]:
import wandb

wandb.init(mode="disabled")

In [22]:
training_args = TrainingArguments(
    output_dir="review_rating_model",
    learning_rate=3e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_spl["train"],
    eval_dataset=tok_spl["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,F1
1,1.051100,0.887286,0.656438
2,0.889400,0.844942,0.659452
3,0.858000,0.836987,0.658493
4,0.835800,0.839565,0.658356
5,0.832900,0.838470,0.659863
6,0.830900,0.833622,0.661370
7,0.829900,0.834105,0.658904


KeyboardInterrupt: 

In [79]:
red_tok_te = tok_te.remove_columns(["text", "index"])
red_tok_te

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 12167
})

In [80]:
b_red_tok_te = red_tok_te.batch(batch_size=64)
b_red_tok_te

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 191
})

In [81]:
preds = list()

In [82]:
model.eval()
model.to(device)
with torch.no_grad():
    for batch in tqdm(b_red_tok_te):
        inputs = data_collator(batch).to(device)
        logits = model(**inputs).logits
        preds.extend(logits.detach().cpu().argmax(dim=1) + 1)

  0%|          | 0/191 [00:00<?, ?it/s]

In [84]:
res_ds = Dataset.from_dict({"index": tok_te["index"], "rate": preds})

In [85]:
res_ds.to_csv("/content/submission.csv")

Creating CSV from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

86237